# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
from skimage.transform import resize
#Library to resize images to a target size.
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import binary_closing, binary_dilation, binary_erosion, binary_opening
from skimage.morphology import selem

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
#Please set the path as per the dataset location.

train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
from skimage.morphology import selem
selem = selem.disk(3)

def min_max_normalization(img):
    return (img - np.min(img)) / (np.max(img) - np.min(img))
  
def crop_center_square(img):
    if img.shape[1] > img.shape[0]:
        offset_l = (img.shape[1] - img.shape[0]) / 2
        offset_r = img.shape[1]-offset_l
        return img[:,int(offset_l):int(offset_r),:]
    elif img.shape[0] > img.shape[1]:
        offset_t = (img.shape[0] - img.shape[1]) / 2
        offset_b = img.shape[0]-offset_t
        return img[int(offset_t):int(offset_b),:,:]
    return img

def centeredCrop(img, new_height, new_width):

   width =  np.size(img,1)
   height =  np.size(img,0)

   left = int(np.ceil((width - new_width)/2.))
   top = int(np.ceil((height - new_height)/2.))
   
   right = int(np.floor((width + new_width)/2.))
   bottom = int(np.floor((height + new_height)/2.))
   
   cImg = img[top:bottom, left:right]
   return cImg

def convert_to_greyscale(img):
    return np.rint(img.mean(axis=-1))

def morpho_transform(img):
    img = binary_erosion(convert_to_greyscale(img))
    return img

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.arange(0,28)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = round(len(folder_list) / batch_size)
        for batch in range(num_batches):
            #print("batch:",batch)# we iterate over the number of batches
            batch_data = np.zeros((batch_size,28,150,150,1)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #print(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    
                    image = crop_center_square(image)
                    image = imresize(image, (150,150))
                    image = min_max_normalization(image)
                    image = morpho_transform(image)
                    #image = binary_erosion(image,selem)
                    
                    batch_data[folder,idx,:,:,0] = image                
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
#Please set the path as per the dataset location.

curr_dt_time = datetime.datetime.now()
train_path = '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense,LSTM, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling2D,MaxPooling3D
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

input_shape = (28,150, 150,1)
num_classes = 5

model = Sequential()

# First convolution layer
model.add(Conv3D(32, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation='relu', input_shape= input_shape))

# Second convolution layer
model.add(Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 2, 2), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
# model.add(Dropout(0.25))

model.add(Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))

model.add(Dense(5, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 26, 148, 148, 32)  896       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 73, 73, 64)    55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 8, 24, 24, 64)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 6, 22, 22, 128)    221312    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 2, 7, 7, 128)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1605760   
__________

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = optimizers.SGD(0.01)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'Conv3D_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(patience=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, 
                     validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  C:\Users\varma\Downloads\Project_data\Project_data\train ; batch size = 10
 C:\Users\varma\Downloads\Project_data\Project_data\val ; batch size = Epoch 1/1510



C:\Anaconda\lib\site-packages\ipykernel\__main__.py:52: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 156s 2s/step - loss: 1.5789 - categorical_accuracy: 0.2701 - val_loss: 1.4911 - val_categorical_accuracy: 0.4200

Epoch 00001: saving model to Conv3D_model_init_2018-10-2410_26_51.071797/model-00001-1.57895-0.27015-1.49108-0.42000.h5
Epoch 2/15
67/67 [==============================] - 142s 2s/step - loss: 1.4655 - categorical_accuracy: 0.3866 - val_loss: 1.3487 - val_categorical_accuracy: 0.4100

Epoch 00002: saving model to Conv3D_model_init_2018-10-2410_26_51.071797/model-00002-1.46547-0.38657-1.34868-0.41000.h5
Epoch 3/15
67/67 [==============================] - 141s 2s/step - loss: 1.2282 - categorical_accuracy: 0.5149 - val_loss: 1.1786 - val_categorical_accuracy: 0.5800

Epoch 00003: saving model to Conv3D_model_init_2018-10-2410_26_51.071797/model-00003-1.22819-0.51493-1.17865-0.58000.h5
Epoch 4/15
67/67 [==============================] - 147s 2s/step - loss: 0.9437 - categorical_accuracy: 0.6358 - val_loss: 0.9287 - val_categorical_accur

In [ ]:
#Reference
'''
CNNLSTM_Model = Sequential()


# define CNN model
CNNLSTM_Model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu')
                                  ,input_shape=input_shape,name = "TimeDist_Conv2D_32")) 

CNNLSTM_Model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),name = "TimeDist_Conv2D_64"))
#CNNLSTM_Model.add(TimeDistributed(BatchNormalization(),name = "batc_1"))
CNNLSTM_Model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2)),name = "TimeDist_MaxPool"))
#CNNLSTM_Model.add(TimeDistributed(Dropout(0.20), name = "Drop_out_1"))

CNNLSTM_Model.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'),name = "TimeDist_Conv2D_128"))
CNNLSTM_Model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2)),name = "TimeDist_MaxPool_2"))


CNNLSTM_Model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),name = "TimeDist_Conv2D_64_2"))


CNNLSTM_Model.add(TimeDistributed(Flatten(),name = "TimeDist_Flatten"))

# define LSTM model
CNNLSTM_Model.add(LSTM(128, return_sequences=True, name = "LSTM_128"))

CNNLSTM_Model.add(LSTM(64, return_sequences=False, name = "LSTM_64_2"))

#CNNLSTM_Model.add(Dense(num_classes, activation='softmax'))

#CNNLSTM_Model.add(GRU(128, return_sequences=True, name = "GRU_128"))
#CNNLSTM_Model.add(GRU(64, return_sequences=False, name = "GRU_64"))

#model.add(Dense(64, activation='relu'))
CNNLSTM_Model.add(Dense(5, activation='softmax'))

print(CNNLSTM_Model.summary())
'''